## TEST

1.請將此測試程式碼，全部接續到HW9的最後部分。

2.部分函式定義在HW9作業裡，所以需先執行HW9的函式碼。

3.若要儲存參數，可在"def train_part345"裡，函式定義最後面加上:

  torch.save(model.state_dict(), "/content/drive/MyDrive/ML2023/midterm/HW9/saveME.pt")
    
  print('Save model weight')


  可參考:https://zhuanlan.zhihu.com/p/82038049


In [ ]:
import h5py  
import numpy as np
import torch
from torch.utils.data import Dataset
#############################################################################
# 模型導入
names = ['resnet47']
name = names[-1]
reset_seed(0)
print(name, '\n')
model = get_resnet(name)
model = model.to(device='cuda')
model.load_state_dict(torch.load('/content/drive/MyDrive/ML2023/midterm/HW9/saveNEW.pt')) # 修改成自己的模型權重存檔位置
model.eval()
#############################################################################
# 讀檔
f = h5py.File('/content/drive/MyDrive/ML2023/midterm/HW9/val.h5','r') # 修改成要測試的數據集
Image1 = f['data']
Label1 = f['labels']

# 定義打包類別
class TestDataset(Dataset):
  def __init__(self, data_tensor, target_tensor):
      self.data_tensor = data_tensor
      self.target_tensor = target_tensor
  def __getitem__(self, index):
      return self.data_tensor[index], self.target_tensor[index]
  def __len__(self):
      # return self.data_tensor.size(0)
      return len(self.data_tensor)

# 分割
Num_dataset = 10
Image1_split = np.split(Image1, Num_dataset)
Label1_split = np.split(Label1, Num_dataset)
Image1_tensor = torch.Tensor( Image1_split )
Label1_tensor = torch.Tensor( Label1_split )
data_combine = TestDataset( Image1_tensor, Label1_tensor )

# 預測
num_correct = 0
num_samples = 0
for x, y in data_combine:
  x = x.to(device='cuda', dtype=to_float)  # move to device, e.g. GPU
  y = y.to(device='cuda', dtype=to_long)
  scores = model(x)
  _, preds = scores.max(1)
  num_correct += (preds == y).sum()
  num_samples += preds.size(0)
acc = float(num_correct) / num_samples
print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
#############################################################################
f.close()